TASK 1 BIG DATA ANALYSIS 

In [1]:
# import SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BigDataAnalysis") \
    .getOrCreate()
#use dataset  
file_path = "C:/Users/sonak/Downloads/Bookings-100000-Rows.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
print(df.show())

+-------------------+-------------------+-------------+--------------------+-----------+------------+-----------------+-------------+-----+-----+--------------------------+------------------------+----------------+-----------------------+-------------+--------------+-------------+--------------+---------------+--------------+
|               Date|               Time|   Booking_ID|      Booking_Status|Customer_ID|Vehicle_Type|  Pickup_Location|Drop_Location|V_TAT|C_TAT|Canceled_Rides_by_Customer|Canceled_Rides_by_Driver|Incomplete_Rides|Incomplete_Rides_Reason|Booking_Value|Payment_Method|Ride_Distance|Driver_Ratings|Customer_Rating|Vehicle Images|
+-------------------+-------------------+-------------+--------------------+-----------+------------+-----------------+-------------+-----+-----+--------------------------+------------------------+----------------+-----------------------+-------------+--------------+-------------+--------------+---------------+--------------+
|2024-07-26 14:0

In [3]:
#check columns
columns_to_check = [
    "Date", "Time", "Booking_ID", "Booking_Status", "Customer_ID",
    "Vehicle_Type", "Pickup_Location", "Drop_Location", "V_TAT", "C_TAT",
    "Canceled_Rides_by_Customer", "Canceled_Rides_by_Driver", "Incomplete_Rides",
    "Incomplete_Rides_Reason", "Booking_Value", "Payment_Method", "Ride_Distance",
    "Driver_Ratings", "Customer_Rating", "Vehicle Images"
]
df = df.dropna(subset=columns_to_check)
df = df.dropDuplicates()

In [4]:
from pyspark.sql.functions import to_date,col
df=df.withColumn("Booking_Date", to_date(col("Date"), "yyyy-MM-dd"))
# Convert numeric columns to float
numeric_cols = ["Booking_Value", "Ride_Distance", "Driver_Ratings", "Customer_Rating"]
for col_name in numeric_cols:
    df=df.withColumn(col_name, col(col_name).cast("float"))

1. Total Booking Value (Revenue)

In [5]:
df.selectExpr("sum(Booking_Value) as Total_Revenue").show()

+-------------+
|Total_Revenue|
+-------------+
|  5.6534614E7|
+-------------+



2. Average Driver Rating and Average Customer Rating

In [6]:
df.selectExpr("avg(Driver_Ratings) as Avg_Driver_Rating", "avg(Customer_Rating) as Avg_Customer_Rating").show()

+------------------+-------------------+
| Avg_Driver_Rating|Avg_Customer_Rating|
+------------------+-------------------+
|3.9974565002397813|  3.998313193677773|
+------------------+-------------------+



3. Top 10 Pickup Locations with Count

In [7]:
df.groupBy("Pickup_Location").count().orderBy("count", ascending=False).show(10)

+----------------+-----+
| Pickup_Location|count|
+----------------+-----+
|    Banashankari| 2201|
|    Yeshwanthpur| 2139|
|        RT Nagar| 2135|
|     Indiranagar| 2133|
|   Sahakar Nagar| 2126|
|    Basavanagudi| 2120|
|Ramamurthy Nagar| 2116|
|     Vijayanagar| 2113|
|     Tumkur Road| 2105|
|        Cox Town| 2100|
+----------------+-----+
only showing top 10 rows



4. Canceled rides by customer and driver

In [8]:
df.groupBy("Canceled_Rides_by_Customer", "Canceled_Rides_by_Driver").count().show()

+--------------------------+------------------------+-----+
|Canceled_Rides_by_Customer|Canceled_Rides_by_Driver|count|
+--------------------------+------------------------+-----+
|                      null|    Customer was coug...| 3654|
|           Change of plans|                    null| 2081|
|             Wrong Address|                    null| 1005|
|      Driver is not mov...|                    null| 3175|
|                      null|    Customer related ...| 5413|
|         AC is Not working|                    null| 1568|
|                      null|    More than permitt...| 2825|
|                      null|    Personal & Car re...| 6542|
|                      null|                    null|74091|
|      Driver asked to c...|                    null| 2670|
+--------------------------+------------------------+-----+



5. Reason for incomplete rides

In [9]:
df.groupBy("Incomplete_Rides_Reason").count().orderBy("count", ascending=False).show()

+-----------------------+-----+
|Incomplete_Rides_Reason|count|
+-----------------------+-----+
|                   null|99098|
|        Customer Demand| 1601|
|      Vehicle Breakdown| 1591|
|            Other Issue|  734|
+-----------------------+-----+



6. Vehicle Type

In [10]:
df.groupBy("Vehicle_Type").count().orderBy("count", ascending=False).show()

+------------+-----+
|Vehicle_Type|count|
+------------+-----+
| Prime Sedan|14877|
|       eBike|14816|
|        Auto|14755|
|  Prime Plus|14707|
|        Bike|14662|
|   Prime SUV|14655|
|        Mini|14552|
+------------+-----+



7. Add monthly and yearly booking count using PySpark

In [12]:
from pyspark.sql.functions import month, year
df = df.withColumn("Month", month(col("Booking_Date")))
df = df.withColumn("Year", year(col("Booking_Date")))
df.groupBy("Year", "Month").count().orderBy("Year", "Month").show()

+----+-----+------+
|Year|Month| count|
+----+-----+------+
|2024|    7|103024|
+----+-----+------+

